# GPU Training Benchmark Suite — Analysis

In [1]:
import json, os, glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

base = 'results'
runs = []
for summary_path in glob.glob(os.path.join(base, '*', '*', 'run_summary.json')):
    with open(summary_path, 'r') as f:
        d = json.load(f)
    d['experiment'] = summary_path.split(os.sep)[1]
    d['run'] = summary_path.split(os.sep)[2]
    runs.append(d)

df = pd.DataFrame(runs)
print('Loaded runs:', len(df))
df.head()

In [2]:
# Throughput vs GPU count (bar)
if not df.empty:
    sns.barplot(data=df, x='world_size', y='total_samples', estimator=sum)
    plt.title('Total Samples by World Size')
    plt.show()

    # If per-epoch JSONL present, aggregate throughput per run
    rows = []
    for _, r in df.iterrows():
        ep_path = os.path.join('results', r['experiment_name'], r['run_id'], 'metrics_epoch.jsonl')
        if os.path.exists(ep_path):
            with open(ep_path) as f:
                for line in f:
                    rows.append({**json.loads(line), 'run_id': r['run_id'], 'world_size': r['world_size']})
    df_ep = pd.DataFrame(rows)
    if not df_ep.empty:
        plt.figure()
        sns.barplot(data=df_ep, x='world_size', y='throughput_samples_per_s')
        plt.title('Throughput (samples/s) by World Size')
        plt.show()

        # Scaling efficiency using median throughput across runs per world size
        eff = (df_ep.groupby('world_size')['throughput_samples_per_s']
                  .median()
                  .rename('thr'))
        if 1 in eff.index:
            base = eff.loc[1]
            eff_df = (eff / (eff.index * base)).reset_index()
            eff_df.columns = ['world_size', 'scaling_efficiency']
            plt.figure()
            sns.lineplot(data=eff_df, x='world_size', y='scaling_efficiency', marker='o')
            plt.ylim(0, 1.1)
            plt.title('Scaling Efficiency')
            plt.show()

        # Step-time distributions (box)
        plt.figure()
        sns.boxplot(data=df_ep, x='world_size', y='p50_step_ms')
        plt.title('Step-time p50 (ms) by World Size')
        plt.show()

        plt.figure()
        sns.boxplot(data=df_ep, x='world_size', y='p95_step_ms')
        plt.title('Step-time p95 (ms) by World Size')
        plt.show()
